## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-09-20-46-56 +0000,nyt,Blue Origin to Launch NASA’s ESCAPADE Mission ...,https://www.nytimes.com/2025/11/09/science/blu...
1,2025-11-09-20-44-39 +0000,bbc,Philippines hit by 'intense' typhoon as nearly...,https://www.bbc.com/news/articles/cq509yj82y8o...
2,2025-11-09-20-35-00 +0000,wsj,BBC Director General and News Chief Resign Aft...,https://www.wsj.com/business/media/bbc-directo...
3,2025-11-09-20-34-06 +0000,startribune,Live: Vikings trail the Ravens 27-13 in the fo...,https://www.startribune.com/minnesota-vikings-...
4,2025-11-09-20-27-13 +0000,latimes,Firefighters free person stuck under E Line train,https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
80,trump,28
97,shutdown,10
116,new,10
187,up,9
16,bbc,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
37,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...,70
20,2025-11-09-19-59-01 +0000,startribune,Trump administration demands states 'undo' ful...,https://www.startribune.com/trump-administrati...,67
157,2025-11-09-02-00-00 +0000,wsj,Big companies run into a roadblock to getting ...,https://www.wsj.com/politics/policy/trump-bide...,64
28,2025-11-09-19-36-13 +0000,cbc,Trump administration wants U.S. states to 'und...,https://www.cbc.ca/news/world/trump-administra...,62
48,2025-11-09-18-12-48 +0000,nyt,Trump Renews Attacks on Obamacare in New Push ...,https://www.nytimes.com/2025/11/09/us/politics...,60


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
37,70,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...
157,36,2025-11-09-02-00-00 +0000,wsj,Big companies run into a roadblock to getting ...,https://www.wsj.com/politics/policy/trump-bide...
67,32,2025-11-09-15-33-10 +0000,nypost,Lefty Gen-Z NYC Councilman Chi Osse gears up f...,https://nypost.com/2025/11/09/us-news/nyc-coun...
43,29,2025-11-09-18-33-00 +0000,nypost,BBC head Tim Davie resigns over doctored foota...,https://nypost.com/2025/11/09/world-news/bbc-h...
49,29,2025-11-09-18-03-36 +0000,nypost,"Crew member dead, 2 critically injured in medi...",https://nypost.com/2025/11/09/us-news/crew-mem...
26,28,2025-11-09-19-40-29 +0000,nyt,"Hamas Hands Over Body of Hadar Goldin, Israeli...",https://www.nytimes.com/2025/11/09/world/middl...
129,27,2025-11-09-09-00-00 +0000,cbc,NDP MP ‘very confident’ no New Democrats will ...,https://www.cbc.ca/news/politics/ndp-rejects-f...
11,24,2025-11-09-20-13-01 +0000,nyt,Super Typhoon Fung-wong Hits Philippines Days ...,https://www.nytimes.com/2025/11/09/world/asia/...
21,24,2025-11-09-19-52-58 +0000,nypost,Fallen Bravest Patrick Brady’s April Fool’s pr...,https://nypost.com/2025/11/09/us-news/fallen-b...
57,24,2025-11-09-17-30-00 +0000,wsj,President Trump suggests tariff revenue could ...,https://www.wsj.com/politics/policy/trump-sugg...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
